# Computer Vision - Project
imports

In [1]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import h5py
import requests
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.utils import np_utils
import tensorflow as tf
import h5py, requests, os
import matplotlib.patches as patches
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)
from keras.utils import plot_model
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Flatten, BatchNormalization, ZeroPadding2D, GlobalMaxPooling2D
from keras.layers.convolutional import Conv2D
from keras.layers.pooling import MaxPooling2D
from keras.layers import LSTM
from keras.layers import Resizing
from tqdm import tqdm

Constants

In [2]:
#ZIP_PATH = "https://drive.google.com/drive/folders/1jzHYpTwywUYA53nMGHVROSuVO14hEueq?usp=sharing/"
FILE_NAME ="SynthText_train.h5"
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
  tf.config.experimental.set_memory_growth(gpu, True)
tf.keras.backend.clear_session()
SIZE=224


In [3]:
def sort_points(points, center):
    # calculate the angle of each point from the center point
    angles = np.arctan2(points[:, 1] - center[1], points[:, 0] - center[0])
    # sort the points by angle
    sorted_points = points[np.argsort(angles)]
    return sorted_points

In [4]:
def draw_training_curve(history):
	"""
	Draw training curve
	Parameters:
		history - contains loss and accuracy from training
	Returns:
		None
	"""
	plt.figure(1)

	# History for accuracy
	plt.subplot(211)
	plt.plot(history.history['accuracy'])
	plt.plot(history.history['val_accuracy'])
	plt.title('model accuracy')
	plt.ylabel('accuracy')
	plt.xlabel('epoch')
	plt.legend(['train', 'test'], loc='upper left')

	# History for loss
	plt.subplot(212)
	plt.plot(history.history['loss'])
	plt.plot(history.history['val_loss'])
	plt.title('model loss')
	plt.ylabel('loss')
	plt.xlabel('epoch')
	plt.legend(['train', 'test'], loc='upper left')

	plt.show()

In [5]:
def draw_points(image, points, color=(255, 0, 0), radius=3):
    # create a copy of the image
    img = image.copy()
    # iterate over the points and draw them on the image
    for point in points:
        cv2.circle(img, tuple(map(int, point)), radius, color, -1)
    return img


In [6]:
def font_to_num(font):
    if font == b'Alex Brush':
        return 0
    elif font == b'Titillium Web':
        return 1
    elif font == b'Sansation':
        return 2
    elif font == b'Open Sans':
        return 3
    else:
        return 4

In [7]:
def num_to_font(font):
    if font == 0:
        return b'Alex Brush'
    elif font == 1:
        return b'Titillium Web'
    elif font == 2:
        return b'Sansation'
    elif font == 3:
        return b'Open Sans'
    else:
        return b'Ubuntu Mono'

In [8]:
def print_label(set, index):
    line = set[index]
    max = np.argmax(line)
    print(num_to_font(max))

In [9]:
def print_font(max):
     print(num_to_font(max))

In [10]:
def print_photo_from_set(set_x, set_y, index, font):
    plt.imshow(set_x[index], cmap='gray')
    plt.show()
    print_font(font)

In [11]:

def prepare_img(img, bbs, index, size):
    x1 = int(bbs[0,0,index])
    y1 = int(bbs[1,0,index])
    x2 = int(bbs[0,1,index])
    y2 = int(bbs[1,1,index])
    x3 = int(bbs[0,2,index])
    y3 = int(bbs[1,2,index])
    x4 = int(bbs[0,3,index])
    y4 = int(bbs[1,3,index])
    # calculate bounding rectangle
    top_left_x = max(0, min([x1,x2,x3,x4]))
    top_left_y = max(0, min([y1,y2,y3,y4]))
    bot_right_x = max(0, max([x1,x2,x3,x4]))
    bot_right_y = max(0, max([y1,y2,y3,y4]))

    cropped = img[top_left_y:bot_right_y+1, top_left_x:bot_right_x+1]
    # _, cropped = cv2.threshold(cropped,127,255,cv2.THRESH_TRIANGLE)
    # print(cropped.shape)
    cropped = tf.image.resize(cropped, (size, size), method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
    # cropped = tf.image.rgb_to_grayscale(cropped)
    # # print(cropped.shape)
    cropped = tf.image.convert_image_dtype(cropped, tf.float32)
    # plt.imshow(cropped, cmap='gray')
    # plt.show()
    return cropped

In [12]:
from pathlib import Path
def create_dirs(main_directory):
    Path(main_directory).mkdir(parents=True, exist_ok=True)
    Path(main_directory+'/Alex Brush').mkdir(parents=True, exist_ok=True)
    Path(main_directory+'/Titillium Web').mkdir(parents=True, exist_ok=True)
    Path(main_directory+'/Sansation').mkdir(parents=True, exist_ok=True)
    Path(main_directory+'/Open Sans').mkdir(parents=True, exist_ok=True)
    Path(main_directory+'/Ubuntu Mono').mkdir(parents=True, exist_ok=True)

In [60]:
def get_image_data(db, im):
    img  = db['data'][im][:]
    fonts = db['data'][im].attrs['font']
    txts = db['data'][im].attrs['txt']
    charBBs = db['data'][im].attrs['charBB']
    wordBBs = db['data'][im].attrs['wordBB']
    return img, fonts, txts, charBBs, wordBBs

In [13]:
def is_num_or_letter(inp):
    # res= ((inp >= ord('a') and inp <= ord('z')) or (inp >= ord('A') and inp <= ord('Z'))) or (inp>=ord('0') and inp<=ord('9'))
    # # print(chr(inp)+"="+str(res))
    if(inp==ord('.') or inp==ord(',') or inp==ord(':') or inp==ord(';')):
        return False
    return True

In [75]:
def crop_and_save(img, BBs, indx, size, curr_font, im, num, append_not_save=False):
    cropped = prepare_img(img, BBs, indx, size)
    path = 'main_directory/'+curr_font.decode('UTF-8')+'/'+im+'_'+str(num)+'.jpg' 
    if not append_not_save:
        tf.keras.utils.save_img(path,cropped)
    return cropped

In [14]:
db = h5py.File(FILE_NAME, 'r')

In [73]:
def get_data_set(db, size: int):   
    create_dirs('main_directory')
    im_names = list(db['data'].keys())
    num = 0
    for i in tqdm(range(0, len(im_names)-1)):
        im = im_names[i]
        img, fonts, txts, charBBs, wordBBs = get_image_data(db, im)
        font_indx = 0 
        char_indx = 0
        # print(im)
        for j in range(0, len(txts)):
            if(j%8!=0):
                crop_and_save(img, wordBBs, j, size, fonts, font_indx, im, num)
                num+=1
                for k in range(0, len(txts[j])):
                    if(is_num_or_letter(txts[j][k])):
                        crop_and_save(img, charBBs, char_indx, size, fonts, font_indx, im, num)
                        num+=1
                    char_indx+=1
            else:
                char_indx+=len(txts[j])
            font_indx += len(txts[j])
    print(num)

In [16]:
from download_h5py_db import download_h5py_db
if not Path("main_directory").exists():
    download_h5py_db()
    get_data_set(db, SIZE)
# train_x, train_y = 

In [17]:
import splitfolders
splitfolders.ratio("main_directory/", # The location of dataset
                   output="main_directory_splitted", # The output location
                   seed=42, # The number of seed
                   ratio=(.8, .1, .1), # The ratio of splited dataset
                   group_prefix=None, # If your dataset contains more than one file like ".jpg", ".pdf", etc
                   move=False # If you choose to move, turn this into True
                   )

Copying files: 29840 files [00:51, 584.48 files/s]


In [18]:
from keras.preprocessing.image import ImageDataGenerator
def data_augmentation():
    datagen =  ImageDataGenerator(
        horizontal_flip=True, rotation_range=90, fill_mode='reflect', channel_shift_range=0.8,#
         shear_range=15,vertical_flip=False, brightness_range=(0.2, 0.8),# 
     rescale=1/255, dtype='float32'
     #,validation_split=0.25
     )
    # Path('augmented').mkdir(exist_ok=True)
    it = datagen.flow_from_directory('main_directory_splitted/train/', batch_size=18, class_mode='categorical',
    #save_to_dir='augmented',
     shuffle=True, seed=1, keep_aspect_ratio=True)
    datagen_val =  ImageDataGenerator(
     rescale=1/255, dtype='float32'
     )
    val_it = datagen_val.flow_from_directory('main_directory_splitted/val/', batch_size=18, class_mode='categorical',
    #save_to_dir='augmented',
     shuffle=True)
    test_it = datagen_val.flow_from_directory('main_directory_splitted/test/', batch_size=18, class_mode='categorical',
    #save_to_dir='augmented',
     shuffle=True)
    return it, val_it,test_it, datagen

In [19]:
it, val_it,test_it, datagen = data_augmentation()

Found 23869 images belonging to 5 classes.
Found 2981 images belonging to 5 classes.
Found 2990 images belonging to 5 classes.


Found 27011 images belonging to 5 classes.
Found 9000 images belonging to 5 classes.

In [20]:
# datagen.fit(it, augment=True, seed=0.8, rounds=5)

In [21]:
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import GaussianNoise

from keras import regularizers

num_classes = 5
input_size= SIZE

baseModel = tf.keras.applications.ResNet50(include_top=False, classes=num_classes,
                         input_shape=(input_size, input_size, 3),
                        weights='imagenet')
headModel = baseModel.output
headModel = GaussianNoise(0.1)(headModel)
headModel = AveragePooling2D(pool_size=(7, 7))(headModel)
headModel = Flatten(name="flatten")(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(256, activation="relu", kernel_regularizer=regularizers.l2(0.01))(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(num_classes, activation="softmax")(headModel)
model = Model(inputs=baseModel.input, outputs=headModel)    
model.compile(optimizer=tf.keras.optimizers.Adam(0.00003), loss='categorical_crossentropy', metrics=[tf.keras.metrics.Precision()])

In [22]:
def scheduler(epoch, lr):
    # if epoch < 5:
    #     return lr
    # else:
    return lr * 0.1 #tf.math.exp(-0.1)

In [23]:
# callback = tf.keras.callbacks.LearningRateScheduler(scheduler)

In [24]:
# model.fit(X_train, Y_train, batch_size=16, epochs=20)
# datagen.fit(X_train)
if not os.path.isfile('model_res/saved_model.pb'):
  history = model.fit(it, epochs=15, shuffle=True, validation_data=val_it, verbose=1
          #   ,callbacks=[callback]
            )
else:
  model = tf.keras.models.load_model('model_res')

In [25]:
score = model.evaluate(test_it)

167/167 [==============================] - 17s 74ms/step - loss: 0.4731 - accuracy: 0.8368


In [26]:
print(f'Test loss: {score[0]} / Test accuracy: {score[1]}')

Test loss: 0.47312721610069275 / Test accuracy: 0.8367893099784851


In [27]:
draw_training_curve(history)

NameError: name 'history' is not defined

In [ ]:
model.save('model_res')

INFO:tensorflow:Assets written to: model_res\assets


INFO:tensorflow:Assets written to: model_res\assets


In [92]:
size=SIZE
im_names = list(db['data'].keys())
num = 0
prediction_arr=[]
test_y = []
test_x = []
for i in tqdm(range(0, int((len(im_names)-1)/4))):
    im = im_names[i]
    img, fonts, txts, charBBs, wordBBs = get_image_data(db, im)
    font_indx = 0 
    char_indx = 0
    curr_font = fonts[font_indx]
    # print(im)
    for j in range(0, int(len(txts)/4)):
        if(j%8==0):  
            cropped = crop_and_save(img, wordBBs, j, size, curr_font, im, num, True)
            test_x.append(cropped)
            test_y.append(font_to_num(curr_font))
            num+=1            
            for k in range(0, len(txts[j])):
                if(is_num_or_letter(txts[j][k])):
                    cropped = crop_and_save(img, charBBs, char_indx, size, curr_font, im, num, True)
                    test_x.append(cropped)
                    test_y.append(font_to_num(curr_font))
                    num+=1
                char_indx+=1
            # test_x = np.asarray(test_x, dtype=np.float32)
            # # print(test_x)
            # reses = model.predict(test_x)
            # # print(reses)
            # maxes = np.argmax(reses, axis=1)
            # prediction = np.bincount(maxes).argmax()
            # for k in range(0, len(txts[j])):
            #     test_y.append(font_to_num(fonts[font_indx]))
            #     prediction_arr.append(prediction)
            # if(fonts[font_indx]==b'Open Sans'):
            # print(txts[j])
            # for imi in test_x:
            #     plt.imshow(imi)
            #     plt.show()
            # print(maxes)
            # print(reses)
            # # print(prediction)
            # print(fonts[font_indx])
            # print(num_to_font(prediction))
            # print(test_x)
            # print(str(np.bincount(maxes).argmax())+"-"+str(font_to_num(fonts[font_indx])))
            # print(np.bincount(maxes).argmax())
        else:
            char_indx+=len(txts[j])
        font_indx += len(txts[j])
print(len(test_x))
print(len(test_y))
test_x = np.asarray(test_x, dtype=np.float32)


100%|██████████| 249/249 [00:02<00:00, 98.93it/s] 


1003
1003


In [90]:
print(len(test_x))

1003


In [82]:
prediction_arr = model.predict(test_x)

32/32 [==============================] - 3s 98ms/step


In [106]:
y_pred=np.argmax(prediction_arr, axis=1)
# y_test=np.argmax(test_y_cat, axis=1)

AxisError: axis 1 is out of bounds for array of dimension 1

In [102]:
test_y = np.asarray(test_y, dtype=np.float32)

In [103]:
test_y_cat = np_utils.to_categorical(test_y, 5)

In [105]:
print(test_y_cat.shape)
print(prediction_arr.shape)

(1003, 5)


AttributeError: 'list' object has no attribute 'shape'

In [108]:
from sklearn.metrics import classification_report
labels=['Alex Brush','Titillium Web','Sansation','Open Sans','Ubuntu Mono']
print(classification_report(test_y_cat, prediction_arr, target_names=labels))

ValueError: Found input variables with inconsistent numbers of samples: [1003, 0]

In [126]:
datagen_t =  ImageDataGenerator(
     rescale=1/255, dtype='float32'
     )
t_it = datagen_t.flow_from_directory('check_test', batch_size=18, class_mode='categorical',shuffle=False
#save_to_dir='augmented',
    )

Found 3850 images belonging to 5 classes.


In [127]:
score = model.evaluate(t_it)

214/214 [==============================] - 15s 68ms/step - loss: 0.6127 - accuracy: 0.7730


In [128]:
predict = model.predict(t_it)

214/214 [==============================] - 14s 64ms/step


In [129]:
# get the class with highest probability for each sample
y_pred = np.argmax(predict, axis=-1)

In [130]:
print(classification_report(t_it.classes, y_pred, target_names=labels))

               precision    recall  f1-score   support

   Alex Brush       0.88      0.91      0.90       818
Titillium Web       0.55      0.60      0.57       547
    Sansation       0.74      0.72      0.73       910
    Open Sans       0.75      0.61      0.68       732
  Ubuntu Mono       0.86      0.95      0.90       843

     accuracy                           0.77      3850
    macro avg       0.76      0.76      0.76      3850
 weighted avg       0.77      0.77      0.77      3850



In [117]:
# prediction_arr_cat = np_utils.to_categorical(prediction_arr, 5)
# test_y_cat = np_utils.to_categorical(test_y, 5)

In [132]:
# import tensorflow as tf

# m = tf.keras.metrics.AUC()

# m.update_state(test_y, prediction_arr) # assuming both have shape (N,)

# r = m.result().numpy()

# print(r)

In [133]:
# y_true = test_y_cat
# y_pred = prediction_arr
# kacc = tf.keras.metrics.Accuracy()
# _ = kacc.update_state(y_true, y_pred)
# print(f'Keras Accuracy acc: {kacc.result().numpy()*100:.3}')

# kbacc = tf.keras.metrics.BinaryAccuracy()
# _ = kbacc.update_state(y_true, y_pred)
# print(f'Keras BinaryAccuracy acc: {kbacc.result().numpy()*100:.3}')

# print(f'SkLearn acc: {accuracy_score(y_true, y_pred)*100:.3}')